In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PowerTransformer
from sklearn.linear_model import LinearRegression,LassoCV,LogisticRegression
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.model_selection import KFold,train_test_split,StratifiedKFold,GridSearchCV,cross_val_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,accuracy_score, \
                            precision_score,recall_score, roc_auc_score
from sklearn.metrics import classification_report
plt.rcParams['font.sans-serif'] = ['simhei']
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
path = './data/cs-training.csv'
df = pd.read_csv(path)
df = df.drop('Unnamed: 0', axis=1)
df.rename(columns = {'SeriousDlqin2yrs':'未来两年可能违约', 'RevolvingUtilizationOfUnsecuredLines':'可用信贷额度比例', 'age':'年龄',
       'NumberOfTime30-59DaysPastDueNotWorse':'逾期30-59天的笔数', 'DebtRatio':'负债率', 'MonthlyIncome':'月收入',
       'NumberOfOpenCreditLinesAndLoans':'信贷数量', 'NumberOfTimes90DaysLate':'逾期90天+的笔数',
       'NumberRealEstateLoansOrLines':'固定资产贷款数', 'NumberOfTime60-89DaysPastDueNotWorse':'逾期60-89天的笔数',
       'NumberOfDependents':'家属数量'},inplace=True)
# isnull()将含有缺失值的表明True False
df.info()
print(df.isnull().sum())
print(df.columns)
df.shape

In [ ]:
# 输出各字段分布情况图
# 大多数字段明显偏态，后续建模需考虑纠偏处理
df.shape
plt.figure(figsize=(20,20),dpi=300)
plt.subplots_adjust(wspace =0.3, hspace =0.3)
for n,i in enumerate(df.columns):
    plt.subplot(4,3,n+1)
    plt.title(i,fontsize=15)
    plt.grid(linestyle='--', alpha=0.5)
    df[i].hist(alpha=0.5)

print(df.shape)


In [ ]:
df.shape

In [ ]:
df.shape
plt.figure(figsize=(20, 20), dpi=300)
plt.subplots_adjust(wspace=0.3, hspace=0.3)
for index,name in enumerate(df.columns):
    plt.subplot(4, 3, index+1)
    plt.title(name, fontsize=15)
    plt.grid(alpha=0.5, linestyle='--')
    df[[name]].boxplot(sym='.')
    

In [ ]:
plt.figure(figsize=(10, 5), dpi=300)
sns.heatmap(df.corr(), cmap='Reds', annot=True)

## 数据预处理

### 异常值处理

In [ ]:
class error_processing():
    def __init__(self, df):
        self.df = df
        
    def show_error(self, df, col, whis=1.5, show=False):
        iqr = df[col].quantile(0.75) - df[col].quantile(0.25)
        upper_bound = df[col].quantile(0.75) + whis * iqr
        lower_bound = df[col].quantile(0.25) - whis * iqr
        print('【', col, '】上界异常值总数：', df[col][df[col] > upper_bound].count())
        if show:
            print('异常值实例：', df[col][df[col] > upper_bound].head(5).T)
        print('【', col, '】下界异常值总数：', df[col][df[col] < lower_bound].count())
        if show:
            print('异常值实例：', df[col][df[col] < lower_bound].head(5).T)
            
    def drop_error(df, col, whis):
        iqr = df[col].quantile(0.75) - df[col].quantile(0.25)
        upper_bound = df[col].quantile(0.75) + whis * iqr
        lower_bound = df[col].quantile(0.25) - whis * iqr
        df = df[(df[col] <= upper_bound) & (df[col] >= lower_bound)]
        return df

n = df.shape[0]
print(n)
ep = error_processing(df)
# 可用信贷额度
# 从分布直方图可知，比例大于1的应该为错误值。
# 错误值共3321，若剔除可能影响建模效果。剔除>=20000的数据
ep.show_error(df, '可用信贷额度比例')
df = df[df.可用信贷额度比例 <= 20000]

# 年龄
# 异常值数量不多，剔除年龄大于100小于18的异常数据
ep.show_error(df, '年龄')
df = df[(df['年龄'] > 18) & (df['年龄'] < 100)]

# 逾期30-59天的笔数
# 根据箱型图去除>80的异常数据
ep.show_error(df, '逾期30-59天的笔数')
df = df[df['逾期30-59天的笔数'] < 80]

# 逾期90天+的笔数
# 根据箱型图去除>80的异常数据
ep.show_error(df, '逾期90天+的笔数')
df = df[df['逾期90天+的笔数'] < 80]

# 逾期60-89天的笔数
# 根据箱型图去除>80的异常数据
ep.show_error(df, '逾期60-89天的笔数')
df = df[df['逾期60-89天的笔数'] < 80]

# 负债率
# 根据箱型图去除>100000的异常数据
ep.show_error(df, '负债率')
df = df[df['负债率'] < 100000]

# 月收入
# 根据箱型图去除>500000的异常数据
ep.show_error(df, '月收入')
df = df[(df['月收入'] < 500000) | df.月收入.isna()]

# 固定资产贷款数
# 根据箱型图去除>20的异常数据
ep.show_error(df, '固定资产贷款数')
df = df[df['固定资产贷款数'] < 20]

# 家属数量
# 根据箱型图去除>10的异常数据
ep.show_error(df, '家属数量')
df = df[(df['家属数量'] <= 12) | df.家属数量.isna()]

print('共删除:', n - df.shape[0], '数据')
df.shape

### 缺失值处理

In [ ]:
def missing_values_processing(df,flag1=1,flag2=1):
    '''
    缺失值处理
    df：数据源
    flag1：默认为1，众数填充家属；0，去除带空值数据行。
    f2：默认为1，众数填充月收入；0，平均数填充月收入。
    '''
    # 家属数量 - 剔除或众数填充
    if flag1 == 1:
        df.loc[df.家属数量.isna(),'家属数量'] = df.家属数量.mode()[0]
    elif flag1 == 2:
        df = df.dropna(subset=['家属数量'])
    else:
        print('parameter wrong!')
    
    # 月收入 - 剔除或均值填充
    if flag2 == 1:
        df.loc[df.月收入.isna(),'月收入'] = df.月收入.mode()[0]
    elif flag2 == 0:    
        df.loc[df.月收入.isna(),'月收入'] = df.月收入.mean()[0]
    else:
        print('parameter wrong!')   

missing_values_processing(df)

### 共线性处

In [ ]:
# 可考虑保留'逾期90天+的笔数'，求出'逾期60-89天的笔数'/'逾期30-59天的笔数'的比值
def collineation_processing(df,col,col1,col2,name):
    '''
    去除共线性，保留一个字段，其他字段求比值
    df：数据源
    col：保留字段
    col1，col2：求比值字段
    name：新比值字段名称
    '''
    def trans2percent(row):
        if row[col2] == 0:
            return 0
        else:
            return row[col1] / row[col2]
    df[name] = df.apply(trans2percent,axis=1)
    print(df[[name,col]].corr())
    

collineation_processing(df, '逾期90天+的笔数', '逾期60-89天的笔数', '逾期30-59天的笔数', '逾期60-89天/30-59天')
df.shape

### 重采样

In [ ]:
# 从数据初探可以发现，'未来两年可能违约'标签类别分布不均，需对样本进行重取样

def resample(df):
    '''
    使样本'未来两年可能违约'标签的0，1项可以各占一半，以提高预测效果。sample()可以考虑添加random_state以便生成相同样本集
    df：数据源
    '''
    num = df['未来两年可能违约'].value_counts()[1]
    df_t = df[df.未来两年可能违约==1]
    df_f = df[df.未来两年可能违约==0].sample(frac=1)[0:num]
    df_balanced = pd.concat([df_t,df_f]).sample(frac=1).reset_index(drop=True)
#     print(df_balanced.未来两年可能违约.value_counts())
    return df_balanced

df = resample(df)


## 得到训练数据

In [ ]:
X = df.drop(['未来两年可能违约', '逾期30-59天的笔数','逾期60-89天的笔数'], axis=1)
y = df['未来两年可能违约']    
xtrain,xtest,ytrain,ytest = train_test_split(X,y,test_size=0.2)    # random_state=42
# 分层k折交叉拆分器 - 用于网格搜索
cv = StratifiedKFold(n_splits=3,shuffle=True)

In [ ]:
# 分类模型性能查看函数
def perfomance_clf(model,X,y,name=None):
    y_predict = model.predict(X)
    if name:
        print(name,':')
    print(f'accuracy score is: {accuracy_score(y,y_predict)}')
    print(f'precision score is: {precision_score(y,y_predict)}')
    print(f'recall score is: {recall_score(y,y_predict)}')
    print(f'auc: {roc_auc_score(y,y_predict)}')
    print(classification_report(y, y_predict, target_names=['未违约', '违约']))
    print('- - - - - - ')

In [ ]:
# 参数设定
log_params = {"penalty":['l1','l2'],
                 'C':[i * 10**i for i in range(0,4)] + [0.1 ** i for i in range(1,4)]}
# 参数搜索
log_gridsearch = GridSearchCV(LogisticRegression(solver='liblinear'),log_params,cv=cv,
                               n_jobs=-1,scoring='roc_auc',verbose=2,refit=True)
# 工作流管道
pipe_log = Pipeline([
        ('sc',StandardScaler()),    # 标准化Z-score
        ('pow_trans',PowerTransformer()),    # 纠偏
        ('log_grid',log_gridsearch)
        ])
# 搜索参数并训练模型
pipe_log.fit(xtrain,ytrain)
# 最佳参数组合
print(pipe_log.named_steps['log_grid'].best_params_)
# 训练集性能指标
perfomance_clf(pipe_log,xtrain,ytrain,name='train')
# 测试集性能指标
perfomance_clf(pipe_log,xtest,ytest,name='test')